In [2]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
df = pd.read_csv("../data/adult.csv")
df = df.replace("?", pd.NA).dropna()
salary_range = ["<=50K", ">50K"]
df = df[df["salary"].isin(salary_range)]
df_salary = pd.to_numeric(df["salary K$"], errors="coerce")
df_salary = df_salary.dropna()
df.head()
# df.info()

# Task 1
Print the count of men and women in the dataset.

In [ ]:
df_men = df[df["sex"] == "Male"]
df_women = df[df["sex"] == "Female"]
df_women.shape[0], df_men.shape[0]

# Task 2
Find the average age of men in dataset

In [ ]:
average_age_men = round(df_men["age"].mean())
average_age_men

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
poland_people = round(df["native-country"].value_counts(normalize=True).get("Poland", 0) * 100, 3)
poland_people

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [ ]:
high_salary = df[df["salary"] == ">50K"]
high_salary_mean_age = round(high_salary["age"].mean())
high_salary_deviation_age = round(high_salary["age"].std())
lower_salary = df[df["salary"] == "<=50K"]
lower_salary_mean_age = round(lower_salary["age"].mean())
lower_salary_deviation_age = round(lower_salary["age"].std())
high_salary_mean_age, high_salary_deviation_age, lower_salary_mean_age, lower_salary_deviation_age

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [ ]:
higher_educations = ["Bachelors", "Prof-school", "Assoc-voc", "Assoc-acdm", "Masters", "Doctorate"]
check_education = high_salary[~high_salary["education"].isin(higher_educations)]
check_education

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
age_statistic = df.groupby("education")["age"].describe()
age_statistic

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [ ]:
married = df[df["marital-status"].str.startswith("Married")]
non_married = df[~df["marital-status"].str.startswith("Married")]
married = married["salary K$"].mean()
non_married = non_married["salary K$"].mean()
compare_salary = pd.DataFrame({
    "Salary range": salary_range,
    "Status": ["Married", "Not Married"],
    "average_salary": [married, non_married]
})
compare_salary


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [ ]:
work_hours = sorted(df["hours-per-week"].unique(), reverse=True)
people_count = (df["hours-per-week"] == work_hours[0]).sum()
work_hours[0], people_count

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [ ]:
data_correlation = df.select_dtypes(include=[np.number])
data_correlation.corr()